In [1]:
import pandas as pd
import numpy as np


pd.set_option('display.max_rows', 999)
import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

import os

In [2]:
file_name = "Relação-de-Compras-COVID19_completa_2804"

dd = camelot.read_pdf(f'../data/orcamento/pdf/{file_name}.pdf', pages='1')

In [4]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [21]:
cols = ['orgao','descricao','valor','n_processo','cnpj','fornecedor','vigencia']
df.columns = cols
df = df[df['descricao']!='']
df['valor'] = df['valor'].apply(lambda x: x.replace('R$','').replace('\n','').strip().replace('.','').replace(',','.')).astype(float)
df['orgao'] = df['orgao'].apply(lambda x: x.replace('\n','').strip())



df['last_update'] = '2020-04-28'

912767442.4499998

In [25]:
print(df['valor'].sum())
print(len(df),"\n")

912767442.4499998
140 



In [28]:
df.to_csv(f'../data/orcamento/csv/{file_name}.csv', index=False)
df.to_excel(f'../data/orcamento/csv/{file_name}.xlsx', index=False)

# Data processing

In [141]:
#Merge all files
path = '../data/orcamento/csv/'
files = os.listdir(path)

df_final = pd.read_csv(f"{path}{files[0]}")
data = max(df_final['last_update'])
df_final[f'valor_{data}'] = df_final['Valor Total']
rename_cols = {'Descrição':f'Descrição_{data}',
               'Vigência (180 dias)':f'Vigência (180 dias)_{data}'} 
df_final = df_final.rename(columns=rename_cols)

for file in files[1:]:
    dff = pd.read_csv(f"{path}{file}")
    print(file)
    print(dff['Valor Total'].sum())
    print(len(dff),"\n")
    
    data = max(dff['last_update'])
    
    dff[f'valor_{data}'] = dff['Valor Total']
    rename_cols = {'Descrição':f'Descrição_{data}',
               'Vigência (180 dias)':f'Vigência (180 dias)_{data}'} 
    dff = dff.rename(columns=rename_cols)

    
    


rel_compras-covid19_publicacao_3v.csv
309661423.25
107 



In [142]:
df_final = pd.merge(df_final, dff, on=['Valor Total','Nº Processo (SES-PRC)','CNPJ'], how='outer')


In [143]:
df_final['Valor Total'].sum()

332681090.38

In [144]:
df_final

,Descrição_2020-04-16,Valor Total,Nº Processo (SES-PRC),CNPJ,Fornecedor_x,Vigência (180 dias)_2020-04-16,last_update_x,valor_2020-04-16,Descrição_2020-04-24,Fornecedor_y,Vigência (180 dias)_2020-04-24,last_update_y,valor_2020-04-24
0,Prestação de Serviços de Telemedicina,3.700000e+06,2020-12.031,60.765.823/0001-30,SOCIEDADE BENEFICIENTE ISRAELITA ALB EINSTEN,13/09/2020,2020-04-16,3700000.00,Prestação de Serviços de Telemedicina,SOCIEDADE BENEFICIENTE ISRAELITA ALB EINSTEN,13/09/2020,2020-04-24,3.700000e+06
1,"Avental Descartavel; Em Nao Tecido, 100% Polip...",4.992000e+06,2020-12.375,02.881.877/0001-64,POLAR FIX IND E COM.PROD.HOSPITALARES LTD,13/09/2020,2020-04-16,4992000.00,"Avental Descartavel; Em Nao Tecido, 100% Polip...",POLAR FIX IND E COM.PROD.HOSPITALARES LTD,13/09/2020,2020-04-24,4.992000e+06
2,"HIGIENIZADOR EM ESPUMA, NEUTRO, COMPOSTO DE AL...",4.368000e+06,2020-12.482,08.304.991/0001-08,PS DISTRIBUIDORA DE PRODUTOS DA SAUDE LTD,13/09/2020,2020-04-16,4368000.00,"HIGIENIZADOREM ESPUMA, NEUTRO,COMPOSTO DE ALCO...",PS DISTRIBUIDORA DE PRODUTOS DA SAUDE LTD,13/09/2020,2020-04-24,4.368000e+06
3,"HIGIENIZADOR EM ESPUMA ANTISSEPTICO DE MAOS, I...",5.600140e+05,2020-12.482,08.304.991/0001-08,PS DISTRIBUIDORA DE PRODUTOS DA SAUDE LTD,13/09/2020,2020-04-16,560014.00,"HIGIENIZADOREM ESPUMAANTISSEPTICODE MAOS,INCOL...",PS DISTRIBUIDORA DE PRODUTOS DA SAUDE LTD,13/09/2020,2020-04-24,5.600140e+05
4,"CAMA HOSPITALAR, COM MOVIMENTOS ELETRICOS:FOWL...",1.380000e+06,2020-12.508,54.178.983/0001-80,HOSPIMETAL IND METALURGICA EQUI,15/09/2020,2020-04-16,1380000.00,"CAMA HOSPITALAR, COM MOVIMENTOS ELETRICOS:FOWL...",HOSPIMETAL IND METALURGICA EQUI,15/09/2020,2020-04-24,1.380000e+06
5,"SERVICO DE LOCACAO, CONTAINER REFRIGERADO",3.082000e+05,2020-12.879,05.411.043/0001-83,CONSTRUMEX CONSTRUÇÃO LTDA,16/09/2020,2020-04-16,308200.00,"SERVICO DE LOCACAO, CONTAINER REFRIGERADO, TEN...",CONSTRUMEX CONSTRUÇÃO LTDA,16/09/2020,2020-04-24,3.082000e+05
6,"OCULOS CIRURGICO DE PROTEÇÃO, EM MATERIAL ACRI...",1.695750e+04,2020-12.919,07.439.329/0003-64,"DVT COMERCIO,IMPORTAÇÃO E EXPORTAÇÃO LTDA",15/09/2020,2020-04-16,16957.50,"OCULOS CIRURGICO DE PROTEÇÃO, EM MATERIAL ACRI...","DVT COMERCIO,IMPORTAÇÃO E EXPORTAÇÃO LTDA",15/09/2020,2020-04-24,1.695750e+04
7,"OCULOS CIRURGICO DE PROTEÇÃO, EM MATERIAL ACRI...",2.431107e+04,2020-12.919,07.439.329/0003-64,"DVT COMERCIO,IMPORTAÇÃO E EXPORTAÇÃO LTDA",15/09/2020,2020-04-16,24311.07,"OCULOS CIRURGICO DE PROTEÇÃO, EM MATERIAL ACRI...","DVT COMERCIO,IMPORTAÇÃO E EXPORTAÇÃO LTDA",15/09/2020,2020-04-24,2.431107e+04
8,"AVENTAL DESCARTAVEL, EM NAO TECIDO, 100% POLIP...",4.750000e+06,2020-12.922,27.608.037/0001-53,DEJAMARO IND E COM DE PROD MED HOSP LTDA,14/09/2020,2020-04-16,4750000.00,"AVENTAL DESCARTAVEL,EM NAO TECIDO,100% POLIPRO...",DEJAMARO IND E COM DE PROD MED HOSP LTDA,14/09/2020,2020-04-24,4.750000e+06
9,"AVENTAL DESCARTAVEL, EM NAO TECIDO (100% POLIP...",3.300000e+06,2020-12.922,27.608.037/0001-53,DEJAMARO IND E COM DE PROD MED HOSP LTDA,14/09/2020,2020-04-16,3300000.00,"AVENTAL DESCARTAVEL, EM NAO TECIDO (100% POLIP...",DEJAMARO IND E COM DE PROD MED HOSP LTDA,14/09/2020,2020-04-24,3.300000e+06


In [98]:
cols = ['Descrição','Valor Total', 'Nº Processo (SES-PRC)', 'CNPJ']


df_final = df_final.drop_duplicates(subset=cols, keep='first')

In [99]:
print(df_final['Valor Total'].sum())
print(len(df_final),"\n")

485974413.63
208 



In [107]:
df_final.sort_values(by='Valor Total').to_excel('lici.xlsx', index=False)

In [25]:
dd = df.groupby(by=['Fornecedor','CNPJ'], as_index=False).sum().sort_values(by='Valor Total', ascending=False).reset_index(drop=True)

# dd.to_excel('../data/orcamento/rel_compras-covid19_fornecedores.xlsx', index=False)

In [26]:
dd

,Fornecedor,CNPJ,Valor Total
0,FORMED REPRESENTAçAO E COM DE EQUIPTOS ME,07.139.218/0001-70,1.044000e+08
1,ILLEC INTERNATIONAL LIMITED - BRASIL,84.451.012/0001-22,6.724029e+07
2,DOMPEL IND. PLASTICA E METALURGICA LTDA.,87.203.121/0001-19,2.580000e+07
3,PS DISTRIBUIDORA DE PRODUTOS DA SAUDE LTD,08.304.991/0001-08,2.159201e+07
4,MARCELO NERES DE OLIVEIRA,28.476.997/0001-70,1.419000e+07
5,DEJAMARO IND E COM DE PROD MED HOSP LTDA,27.608.037/0001-53,1.216950e+07
6,TECNO 4 PRODUTOS HOSPITALARES LTDA. EPP,04.124.669/0001-46,1.104500e+07
7,CIR.FERNANDES COM.MAT.CIR.E HOS.SOCIED.LT,61.418.042/0001-31,9.823441e+06
8,3M DO BRASIL LTDA,45.985.371/0001-08,7.719990e+06
9,CIRURGICA KD LTDA,09.260.071/0001-06,6.562600e+06
